In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import chardet
import nltk
nltk.download('vader_lexicon')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
from transformers import BertTokenizer, BertModel
from transformers import get_linear_schedule_with_warmup
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\13793\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\13793\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Clean the raw data

In [17]:
data = pd.read_csv('C:/Users/13793/Desktop/aas/stock/twitter.csv', encoding='ISO-8859-1')
data['Text'] = data['Text'].str.lower()

## FIX HYPERLINKS
data['Text'] = data['Text'].replace(r'https?:\/\/.*[\r\n]*', ' ',regex=True)
data['Text'] = data['Text'].replace(r'www.*[\r\n]*', ' ',regex=True)
data['Text'] = data['Text'].str.replace('https', '', regex=False)


	## FIX INDIVIDUAL SYMBOLS 
data['Text'] = data['Text'].str.replace(': ', ' ', regex=False)
data['Text'] = data['Text'].str.replace(', ', ' ', regex=False)
data['Text'] = data['Text'].str.replace('. ', ' ', regex=False)
data['Text'] = data['Text'].str.replace('[;\n~]', ' ', regex=True)

data['Text'] = data['Text'].str.replace("[]'â€¦*™|]", '', regex=True)
data['Text'] = data['Text'].str.replace('[[()!?"]', '', regex=True)
data['Text'] = data['Text'].str.replace('_', '', regex=False)
data['Text'] = data['Text'].str.replace('w/', ' with ', regex=False)
data['Text'] = data['Text'].str.replace('f/', ' for ', regex=False)


	## FIX EMOJIS
data['Text'] = data['Text'].str.replace(':)', '', regex=False)
data['Text'] = data['Text'].str.replace(':-)', '', regex=False)
data['Text'] = data['Text'].str.replace(':(', '', regex=False)
data['Text'] = data['Text'].str.replace(':-(', '', regex=False)
data['Text'] = data['Text'].str.replace('0_o', '', regex=False)
data['Text'] = data['Text'].str.replace(';)', '', regex=False)
data['Text'] = data['Text'].str.replace('=^.^=', '', regex=False)


	## FIX % SYMBOL
data['Text'] = data['Text'].str.replace('%', ' percent ', regex=False)


	## FIX & SYMBOL
data['Text'] = data['Text'].str.replace(' & ', ' and ', regex=False)
data['Text'] = data['Text'].str.replace('&amp', ' and ', regex=False)
data['Text'] = data['Text'].str.replace('&gt', ' greater than ', regex=False)
data['Text'] = data['Text'].str.replace('cup&handle', 'cup and handle', regex=False)
data['Text'] = data['Text'].str.replace('c&h', 'cup and handle', regex=False)
data['Text'] = data['Text'].str.replace('head&shoulders', 'head and shoulders', regex=False)
data['Text'] = data['Text'].str.replace('h&s', 'head and shoulders', regex=False)
data['Text'] = data['Text'].str.replace('point&figure', 'point and figure', regex=False)
data['Text'] = data['Text'].str.replace('p&f', 'point and figure', regex=False)
data['Text'] = data['Text'].str.replace('s&p', 'SP500', regex=False)
data['Text'] = data['Text'].str.replace('q&a', 'question and answer', regex=False)
data['Text'] = data['Text'].str.replace('&', ' and ', regex=False)


	## FIX USER TAGS AND HASTAGS
data['Text'] = data['Text'].str.replace('@[a-z0-9]+', '', regex=True)
data['Text'] = data['Text'].str.replace('#[a-z0-9]+', '', regex=True)
data['Text'] = data['Text'].str.replace('@', '', regex=False)
data['Text'] = data['Text'].str.replace('#', '', regex=False)
	   
		
	## FIX EMBEDDED COMMAS AND PERIODS    
data['Text'] = data['Text'].replace(r'([a-z]),([a-z])', r'\1 \2', regex=True)
data['Text'] = data['Text'].replace(r'([0-9]),([0-9])', r'\1\2', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])[+]+', r'\1 ', regex=True)
data['Text'] = data['Text'].str.replace(',', '', regex=False)
data['Text'] = data['Text'].str.replace('u.s.', ' us ', regex=False)
data['Text'] = data['Text'].str.replace('\.{2,}', ' ', regex=True)
data['Text'] = data['Text'].replace(r'([a-z])\.([a-z])', r'\1 \2', regex=True)
data['Text'] = data['Text'].str.replace('pdating', 'updating', regex=False) 
data['Text'] = data['Text'].replace(r'([a-z])\.', r'\1 ', regex=True)
data['Text'] = data['Text'].replace(r'\.([a-z])', r' \1', regex=True)
data['Text'] = data['Text'].str.replace(' . ', ' ', regex=False)
		

	## FIX + SYMBOL
data['Text'] = data['Text'].replace(r'[+]([0-9])', r'positive \1', regex=True)
data['Text'] = data['Text'].str.replace('c+h', 'cup and handle', regex=False)
data['Text'] = data['Text'].str.replace('h+s', 'head and shoulders', regex=False)
data['Text'] = data['Text'].str.replace('cup+handle', 'cup and handle', regex=False)
data['Text'] = data['Text'].str.replace(' + ', ' and ', regex=False)
data['Text'] = data['Text'].str.replace('+ ', ' ', regex=False)
data['Text'] = data['Text'].replace(r'([a-z])[+]([a-z])', r'\1 and \2', regex=True)
data['Text'] = data['Text'].str.replace('+', '', regex=False)



		
	## FIX - SYMBOL
data['Text'] = data['Text'].replace(r'([a-z])[-]+([a-z])', r'\1 \2', regex=True)
data['Text'] = data['Text'].replace(r'([a-z]) - ([a-z])', r'\1 to \2', regex=True)
data['Text'] = data['Text'].replace(r'([0-9]) -([0-9\.])', r'\1 to \2', regex=True)
data['Text'] = data['Text'].replace(r' [-]([0-9])', r' negative \1', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])-([0-9\.])', r'\1 to \2', regex=True)
data['Text'] = data['Text'].replace(r'([0-9]) - ([0-9\.])', r'\1 to \2', regex=True)
data['Text'] = data['Text'].replace(r'([0-9a-z])-([0-9a-z])', r'\1 \2', regex=True)
data['Text'] = data['Text'].str.replace('[-]+[>]', ' ', regex=True)
data['Text'] = data['Text'].str.replace(' [-]+ ', ' ', regex=True)
data['Text'] = data['Text'].str.replace('-', ' ', regex=False)



	## FIX $ SYMBOL
data['Text'] = data['Text'].str.replace('[$][0-9\.]', ' dollars ', regex=True)
data['Text'] = data['Text'].str.replace('$', '', regex=False)


	## FIX = SYMBOL
data['Text'] = data['Text'].str.replace('=', ' equals ', regex=False)

		
	## FIX / SYMBOL
data['Text'] = data['Text'].str.replace('b/c', ' because ', regex=False)
data['Text'] = data['Text'].str.replace('b/out', ' break out ', regex=False)
data['Text'] = data['Text'].str.replace('b/o', ' break out ', regex=False)
data['Text'] = data['Text'].str.replace('p/e', ' pe ratio ', regex=False)
data['Text'] = data['Text'].str.replace(' [/]+ ', ' ', regex=True)
data['Text'] = data['Text'].str.replace(' 1/2 ', ' .5 ', regex=False)
data['Text'] = data['Text'].str.replace(' 1/4 ', ' .25 ', regex=False)
data['Text'] = data['Text'].str.replace(' 3/4 ', ' .75 ', regex=False)
data['Text'] = data['Text'].str.replace(' 1/3 ', ' .3 ', regex=False)
data['Text'] = data['Text'].str.replace(' 2/3 ', ' .6 ', regex=False)

data['Text'] = data['Text'].str.replace('[/]{2,}', ' ', regex=True)
data['Text'] = data['Text'].replace(r'([a-z])/([a-z])', r'\1 and \2', regex=True)
data['Text'] = data['Text'].str.replace('[0-9]+/[0-9]+/[0-9]+', '', regex=True)
data['Text'] = data['Text'].replace(r'([0-9]{3,})/([0-9\.]{2,})', r'\1 to \2', regex=True)	
data['Text'] = data['Text'].replace(r'([0-9]{2,})/([0-9\.]{3,})', r'\1 to \2', regex=True)
data['Text'] = data['Text'].str.replace('[a-z0-9]+/[a-z0-9]+', ' ', regex=True)

data['Text'] = data['Text'].str.replace('/', '', regex=False)


	## FIX < > SYMBOLS
data['Text'] = data['Text'].str.replace('[<]+ ', ' ', regex=True)
data['Text'] = data['Text'].str.replace('<', ' less than ', regex=False)
data['Text'] = data['Text'].str.replace(' [>]+', ' ', regex=True)
data['Text'] = data['Text'].str.replace('>', ' greater than ', regex=False)


	## FIX : SYMBOL
data['Text'] = data['Text'].str.replace('[0-9]+:[0-9]+am', ' ', regex=True)
data['Text'] = data['Text'].str.replace('[0-9]+:[0-9]', ' ', regex=True)
data['Text'] = data['Text'].str.replace(':', ' ', regex=False)


	## FIX UNITS
data['Text'] = data['Text'].str.replace('user ', ' ', regex=False)

data['Text'] = data['Text'].replace(r'([0-9]+)dma', r'\1 displaced moving average ', regex=True)
data['Text'] = data['Text'].replace(r'dma([0-9]+)', r'\1 displaced moving average ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9]+)sma', r'\1 simple moving average ', regex=True)
data['Text'] = data['Text'].replace(r'sma([0-9]+)', r'\1 simple moving average ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9]+)ema', r'\1 expontential moving average ', regex=True)
data['Text'] = data['Text'].replace(r'ema([0-9]+)', r'\1 expontential moving average ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9]+)ma', r'\1 moving average ', regex=True)
data['Text'] = data['Text'].replace(r'ma([0-9]+)', r'\1 moving average ', regex=True)

data['Text'] = data['Text'].replace(r'([0-9])mos', r'\1 months ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])minute', r'\1 minute ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])minutes', r'\1 minutes ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])min', r'\1 minute ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])mins', r'\1 minutes ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])day', r'\1 day ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])days', r'\1 days ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])wk', r'\1 week ', regex=True)
data['Text'] = data['Text'].str.replace(' wk ', ' week ', regex=False)
data['Text'] = data['Text'].str.replace(' wknd ', ' weekend ', regex=False)
data['Text'] = data['Text'].replace(r'([0-9])wks', r'\1 weeks ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])hours', r'\1 hours ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])hour', r'\1 hour ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])yr', r'\1 year ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])yrs', r'\1 years ', regex=True)
data['Text'] = data['Text'].str.replace(' yr', ' year ', regex=False)
data['Text'] = data['Text'].replace(r'([0-9])am', r'\1 am ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])pm', r'\1 pm ', regex=True)

data['Text'] = data['Text'].replace(r'([0-9])est', r'\1 ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])ish', r'\1 ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9 ])pts', r'\1 points ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])x', r'\1 times ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])th', r'\1 ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])rd', r'\1 ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])st', r'\1 ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])nd', r'\1 ', regex=True)

data['Text'] = data['Text'].str.replace('mrkt', 'market', regex=False)
data['Text'] = data['Text'].str.replace(' vol ', ' volume ', regex=False)
data['Text'] = data['Text'].str.replace(' ptrend', ' positive trend ', regex=False)
data['Text'] = data['Text'].str.replace(' ppl', ' people ', regex=False)
data['Text'] = data['Text'].str.replace(' pts', ' points ', regex=False)
data['Text'] = data['Text'].str.replace(' pt', ' point ', regex=False)
data['Text'] = data['Text'].str.replace(' l(ol){1,}', ' laugh ', regex=True)
data['Text'] = data['Text'].str.replace('imho', ' in my opinion ', regex=True)
data['Text'] = data['Text'].str.replace('prev ', 'previous ', regex=True)


data['Text'] = data['Text'].str.replace(' 1q', ' first quarter ', regex=False)
data['Text'] = data['Text'].str.replace(' 2q', ' second quarter ', regex=False)
data['Text'] = data['Text'].str.replace(' 3q', ' third quarter ', regex=False)
data['Text'] = data['Text'].str.replace(' 4q', ' fourth quarter ', regex=False)
data['Text'] = data['Text'].str.replace(' q1', ' first quarter ', regex=False)
data['Text'] = data['Text'].str.replace(' q2', ' second quarter ', regex=False)
data['Text'] = data['Text'].str.replace(' q3', ' third quarter ', regex=False)
data['Text'] = data['Text'].str.replace(' q4', ' fourth quarter ', regex=False)
data['Text'] = data['Text'].str.replace(' 10q ', ' form 10 ', regex=False)

data['Text'] = data['Text'].replace(r'([0-9])million', r'\1 million ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])mil', r'\1 million ', regex=True)
data['Text'] = data['Text'].str.replace(' mil ', ' million ', regex=False)
data['Text'] = data['Text'].replace(r'([0-9])billion', r'\1 billion ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])cents', r'\1 cents ', regex=True)

data['Text'] = data['Text'].replace(r'([0-9])3d', r'\1 3 dimensional ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])gb', r'\1 3 gigabytes ', regex=True)



data['Text'] = data['Text'].replace(r'([0-9])c', r'\1 calls ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])y', r'\1 year ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])p', r'\1 puts ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])d', r'\1 days ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])h', r'\1 hour ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])s', r'\1 ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])k1', r'\1 thousand ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])k', r'\1 thousand ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])m', r'\1 million ', regex=True)
data['Text'] = data['Text'].replace(r'([0-9])b', r'\1 billion ', regex=True)

		
data['Text'] = data['Text'].replace(r'([0-9])([a-z])', r'\1 \2', regex=True)

	## FIX EXTRA SPACES AND ENDING PUNCTUATION
data['Text'] = data['Text'].str.replace(' +', ' ', regex=True)
data['Text'] = data['Text'].str.strip(' .!?,)(:-')


Classification the positive/negative sentiment

In [5]:
#Clear up meaningless words for NB
Stop = set([s.replace("'", '') for s in stopwords.words('english') if s not in ['not', 'up', 'down', 'above', 'below', 'under', 'against','no','shouldnt']])
data['Text'] = data['Text'].apply(lambda s: " ".join([word for word in s.split() if word not in Stop]))
data['Text'] = data['Text'].str.strip()

In [6]:

model = SentimentIntensityAnalyzer()
#baseline
data['V_score'] = data['Text'].apply(lambda score: model.polarity_scores(score)['compound'])
data['V_prediction'] = data['V_score'].apply(lambda score: 1 if score>=0 else -1)
print('Vader Accuracy:', round((len(data[data['Sentiment']==data['V_prediction']])/len(data)) *100, 2), '%', '\n')

Vader Accuracy: 66.14 % 



In [7]:
#Naive Bayesian Train/Test
train_pct = .8
np.random.seed(100)
idx = np.random.permutation(len(data))

X_train = data['Text'].values[idx[:int(train_pct*len(data))]]
y_train = data['Sentiment'].values[idx[:int(train_pct*len(data))]]
y_train[y_train==-1] = 0
X_test = data['Text'].values[idx[int(train_pct*len(data)):]]
y_test = data['Sentiment'].values[idx[int(train_pct*len(data)):]]
y_test[y_test==-1] = 0


# Calculate TF-IDF for Naive Bayes classification
tf_idf = TfidfVectorizer(ngram_range=(1, 3),
                         binary=True,
                         smooth_idf=False)

# Get TF-IDF for Train and Test data
X_train_tfidf = tf_idf.fit_transform(X_train)
X_test_tfidf = tf_idf.transform(X_test)


# Define function to determine accuracy of model
def get_auc_CV(model):
    kf = StratifiedKFold(5, shuffle=True, random_state=1)

    auc = cross_val_score(model, X_train_tfidf, y_train, scoring="roc_auc", cv=kf)

    return auc.mean()

# Find best performing model
alphas = np.arange(0.4,1.5,0.1)
models = [MultinomialNB(alpha=i) for i in alphas]
accs = []
for model in models:
    accs.append(get_auc_CV(model))
accs = np.array(accs)
best_alpha = round(alphas[accs.argmax()], 1)

# Print best alpha value and accuracy
print('Best alpha: ', best_alpha, '  |  Best Score: ', round(accs.max()*100, 2))

# Retrain best performing model
best_model = MultinomialNB(alpha=best_alpha)
best_model.fit(X_train_tfidf, y_train)

# Predict test data with best model
probs = best_model.predict_proba(X_test_tfidf)

# Print accuracy of best performing model on tweet sentiment analysis 
print('Naive-Bayes Accuracy:', round(len(np.where(y_test == probs.argmax(axis=1))[0])/len(probs) * 100, 2), '%')

Best alpha:  0.6   |  Best Score:  84.35
Naive-Bayes Accuracy: 71.7 %


In [8]:
#K-Fold
kfold = KFold(n_splits=5,
                 shuffle=True,
                 random_state=132)
mses = np.zeros((1, 5))
i=0
for train_index, test_index in kfold.split(data):
    X_train = data.loc[train_index]['Text']
    y_train = data.loc[train_index]['Sentiment']
    y_train[y_train==-1] = 0
    X_test = data.loc[test_index]['Text']
    y_test = data.loc[test_index]['Sentiment']
    y_test[y_test==-1] = 0
    # Calculate TF-IDF for Naive Bayes classification
    tf_idf = TfidfVectorizer(ngram_range=(1, 3),
                         binary=True,
                         smooth_idf=False)

    # Get TF-IDF for Train and Test data
    X_train_tfidf = tf_idf.fit_transform(X_train)
    X_test_tfidf = tf_idf.transform(X_test)
 
    best_model = MultinomialNB(alpha=0.6)
    best_model.fit(X_train_tfidf, y_train)
    probs = best_model.predict_proba(X_test_tfidf)
    
    
    
    mses[0,i] = round(len(np.where(y_test == probs.argmax(axis=1))[0])/len(probs) * 100, 2)
    i=i+1
print('Naive-Bayes Accuracy (K-Fold):', mses.mean(), '%')

Naive-Bayes Accuracy (K-Fold): 71.804 %


Bert Sentiment Analysis

In [18]:
#Train Test Split
train_pct = .8
np.random.seed(10)
idx = np.random.permutation(len(data))

X_train = data['Text'].values[idx[:int(train_pct*len(data))]]
y_train = data['Sentiment'].values[idx[:int(train_pct*len(data))]]
y_train[y_train==-1] = 0
X_test = data['Text'].values[idx[int(train_pct*len(data)):]]
y_test = data['Sentiment'].values[idx[int(train_pct*len(data)):]]
y_test[y_test==-1] = 0

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

In [24]:
#BERT processing function
def prep(data):
    input_ids = []
    attention_masks = []
    
    for i in data:
        encoding = tokenizer.encode_plus(
                text=i, 
                add_special_tokens=True, # adds special chars [CLS] and [SEP] to encoding 
                padding='max_length', # pad the tweets with 0s to fit max length
                max_length = MAX_LEN, # assign max length
                truncation=True, 
                return_tensors="pt", 
                return_attention_mask=True )

        # add the encodings to the list
        input_ids.append(encoding.get('input_ids'))
        attention_masks.append(encoding.get('attention_mask'))
    
    # return the lists as tensors
    input_ids = torch.concat(input_ids)
    attention_masks = torch.concat(attention_masks)
    
    return input_ids, attention_masks

In [25]:
#Setting up for BERT
encoded = [tokenizer.encode(i, add_special_tokens=True) for i in data['Text'].values]
MAX_LEN = max([len(i) for i in encoded])
print('Max length: ', MAX_LEN)
X_train_inputs, X_train_masks = prep(X_train)
X_test_inputs, X_test_masks = prep(X_test)

y_train_labels = torch.tensor(y_train)
y_test_labels = torch.tensor(y_test)

batch_size = 16

train_data = TensorDataset(X_train_inputs, X_train_masks, y_train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(X_test_inputs, X_test_masks, y_test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

Max length:  53


In [27]:
# Define the Bert NLP Classifier
class BertClassifier(nn.Module):
    def __init__(self, freeze=False):
        super(BertClassifier, self).__init__()
        input_layer = 768
        hidden_layer = 50
        output_layer = 2

        # Use the pretrained Bert model for first section of NN
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Define a final layer to attach to the Bert model for custom classification
        self.classifier = nn.Sequential(
            nn.Linear(input_layer, hidden_layer), 
            nn.ReLU(), 
            nn.Linear(hidden_layer, output_layer))

        # Freeze the model from updating
        if freeze:
            for i in self.bert.parameters():
                i.requires_grad = False
        
    # Return classification from Bert model 
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask)
        layer = outputs[0][:, 0, :]
        logits = self.classifier(layer)

        return logits
    
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
np.random.seed(1)

# Check if GPU is available and assign device 
if torch.cuda.is_available():       
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
model = BertClassifier(freeze=False)
model.to(device)

# Define model hyperparameters
epochs = 4
steps = len(train_dataloader) * epochs
lr = 5e-5
eps = 1e-8
optimizer = AdamW(model.parameters(), lr=lr, eps=eps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)
loss_function = nn.CrossEntropyLoss()

# For the number of epochs
for e in range(epochs):
    # Assign model to train
    model.train()

    # Intialize loss to zero
    train_loss = 0
    
    # For each batch
    for batch in train_dataloader:
        batch_inputs, batch_masks, batch_labels = batch
        batch_inputs = batch_inputs.to(device)
        batch_masks = batch_masks.to(device)
        batch_labels = batch_labels.to(device)

        # Reset the model gradient
        model.zero_grad()

        # Get classification of encoded values
        logits = model(batch_inputs, batch_masks)
        
        # Calculate loss based on predictions and known values
        loss = loss_function(logits, batch_labels)
        train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over batch
    train_loss /= len(train_dataloader)
    
    # Assign the model to evaluate    
    model.eval()

    # Initialize losses
    test_loss = 0
    test_acc = 0

    for batch in test_dataloader:
        batch_inputs, batch_masks, batch_labels = batch
        batch_inputs = batch_inputs.to(device)
        batch_masks = batch_masks.to(device)
        batch_labels = batch_labels.to(device)
        with torch.no_grad():
            logits = model(batch_inputs, batch_masks)
        loss = loss_function(logits, batch_labels)
        test_loss += loss.item()

        # Convert predictions to 0 and 1
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate accuracy of model on test data; NOTICE: this accuracy function has to align with previous models
        accuracy = (preds == batch_labels).cpu().numpy().mean() * 100
        test_acc += accuracy

    # Calculate average loss and accuracy per each batch
    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)

    print('Epoch: %d  |  Train Loss: %1.5f  |  Test Loss: %1.5f  |  Test Accuracy: %1.2f'%(e+1, train_loss, test_loss, test_acc))
    
torch.save(model.state_dict(), 'stock_sentiment_model.pt')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

c:\Users\13793\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch: 1  |  Train Loss: 0.53139  |  Test Loss: 0.42354  |  Test Accuracy: 79.38
Epoch: 2  |  Train Loss: 0.32014  |  Test Loss: 0.39786  |  Test Accuracy: 82.08
Epoch: 3  |  Train Loss: 0.15164  |  Test Loss: 0.62618  |  Test Accuracy: 81.63
Epoch: 4  |  Train Loss: 0.06619  |  Test Loss: 0.75469  |  Test Accuracy: 83.13
